In [16]:
import pandas as pd
import pyodbc
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px

from matplotlib.backends.backend_pdf import PdfPages
import matplotlib
matplotlib.use('Agg')
pdf_out = PdfPages('wrapped_charts.pdf')

server = 'projectmbappe.database.windows.net'
database = 'Spotify'
username = 'shazayub774'
password = 'Ntqmskw4?'
driver = 'ODBC Driver 17 for SQL Server'  # This may differ based on your SQL Server driver

conn = pyodbc.connect(f"DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}")
conn_str = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver={driver}'

engine = create_engine(conn_str)

query = 'SELECT * FROM plays_list'
query2 = 'SELECT * FROM track_list'

plays_list = pd.read_sql(query, engine)
track_list = pd.read_sql(query2, engine)

artists = plays_list.groupby('artist').size().reset_index(name ='No. of times played')
most_played = pd.DataFrame(artists).sort_values(by = 'No. of times played', ascending = False)
top_30 = most_played.head(30)

tracks = plays_list.groupby('track').size().reset_index(name ='No. of times played')
most_played_t = pd.DataFrame(tracks).sort_values(by = 'No. of times played', ascending = False)
top_30_t = most_played_t.head(30)

group = plays_list.groupby('track_id').size().reset_index(name ='No. of times played')
mp = group.sort_values(by = 'No. of times played', ascending = False)
mp_tracks = pd.merge(mp, track_list, on = 'track_id', how = 'left')
top_tracks = mp_tracks.head(30)

In [17]:
# i want to see when i listened to the most music so i created a histogram using date played
plt.figure(figsize=(20,9))
plt.hist(plays_list['date_played'], bins = 108) #108 is a rough estimate to make bins a month wide #int(np.sqrt(len(plays_list['date_played']))))
plt.xlabel('Date')
plt.ylabel('Frequency')
plt.title('Listening Frequency vs Month', fontweight='bold', fontsize=20)
# i want the histogram x axis to read easier. i want it to create a label every 6 months
min_date = plays_list['date_played'].min()
max_date = plays_list['date_played'].max()
six_month_intervals = pd.date_range(start = min_date, end = max_date, freq = '6MS')
plt.xticks(six_month_intervals, [date.strftime('%b %Y') for date in six_month_intervals], rotation = 45)
plt.tight_layout()
pdf_out.savefig()
plt.show()

C:\Users\shaz_\AppData\Local\Temp\ipykernel_35444\2084425786.py:14: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [19]:
plt.figure(figsize=(15,9))
bars = plt.barh(top_tracks['track'], top_tracks['No. of times played'])
plt.title('Most Played Tracks', fontweight='bold', fontsize=20)
plt.xlabel('No. of times played')
plt.bar_label(bars, labels=top_tracks['No. of times played'], label_type='edge', fontsize=10, padding = 5)
plt.gca().invert_yaxis()
plt.yticks(ticks=top_tracks['track'], labels=top_tracks['track'] + " | " + top_tracks['artist'])
plt.tight_layout()
pdf_out.savefig()
plt.show()

C:\Users\shaz_\AppData\Local\Temp\ipykernel_35444\2919579339.py:10: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [20]:
# i want to visualise this so i'm creating a bar chart
#top_30x = top_30[::-1]
plt.figure(figsize=(15,9))
bars = plt.barh(top_30['artist'], top_30['No. of times played'])
plt.title('Most Played Artists', fontweight='bold', fontsize=20)
plt.xlabel('No. of times played')
plt.bar_label(bars, labels=top_30['No. of times played'], label_type='edge', fontsize=10, padding = 5)
plt.gca().invert_yaxis()
plt.tight_layout()
pdf_out.savefig()
plt.show()

C:\Users\shaz_\AppData\Local\Temp\ipykernel_35444\3964534291.py:11: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [21]:
# https://python-graph-gallery.com/circular-barplot-basic/

plt.figure(figsize=(25,20))
ax=plt.subplot(111, polar=True)
plt.axis('off')
plt.title('Most Played Tracks', fontweight='bold', fontsize=20)


upperLimit = 100
lowerLimit = 30

max = top_tracks['No. of times played'].max()

slope = (max - lowerLimit)/max
heights = slope * top_tracks['No. of times played'] + lowerLimit

width = 2*np.pi / len(top_tracks.index)

indexes = list(range(1, len(top_tracks.index)+1))
angles = [element * width for element in indexes]

bars = ax.bar(
    x = angles,
    height = heights,
    width = width,
    bottom = lowerLimit,
    linewidth = 2,
    edgecolor = 'white')

# little space between the bar and the label
labelPadding = 4

# Add labels
for bar, angle, height, label in zip(bars,angles, heights, top_tracks['track']+ ' -- ' +top_tracks['artist']):

    # Labels are rotated. Rotation must be specified in degrees :(
    rotation = np.rad2deg(angle)

    # Flip some labels upside down
    alignment = ""
    if angle >= np.pi/2 and angle < 3*np.pi/2:
        alignment = "right"
        rotation = rotation + 180
    else: 
        alignment = "left"

    # Finally add the labels
    ax.text(
        x=angle, 
        y=lowerLimit + bar.get_height() + labelPadding, 
        s=label, 
        ha=alignment, 
        va='center', 
        rotation=rotation, 
        rotation_mode="anchor") 
    
for bar, angle, height, label in zip(bars,angles, heights, top_tracks['No. of times played']):

    # Labels are rotated. Rotation must be specified in degrees :(
    rotation = np.rad2deg(angle)

    # Flip some labels upside down
    alignment = ""
    if angle >= np.pi/2 and angle < 3*np.pi/2:
        alignment = "right"
        rotation = rotation + 180
    else: 
        alignment = "left"

    # Finally add the labels
    ax.text(
        x=angle, 
        y= 125,
        s=label, 
        ha=alignment, 
        va='center', 
        rotation=rotation, 
        rotation_mode="anchor",
        color = 'white',
        weight = 'heavy')

pdf_out.savefig()

In [22]:
#ttps://python-graph-gallery.com/circular-barplot-basic/

plt.figure(figsize=(18,18))
ax=plt.subplot(111, polar=True)
plt.axis('off')
plt.title('Most Played Artists', fontweight='bold', fontsize=20)


upperLimit = 100
lowerLimit = 30

max = top_30['No. of times played'].max()

slope = (max - lowerLimit)/max
heights = slope * top_30['No. of times played'] + lowerLimit

width = 2*np.pi / len(top_30.index)

indexes = list(range(1, len(top_30.index)+1))
angles = [element * width for element in indexes]

bars = ax.bar(
    x = angles,
    height = heights,
    width = width,
    bottom = lowerLimit,
    linewidth = 2,
    edgecolor = 'white')

# little space between the bar and the label
labelPadding = 50

# Add labels
for bar, angle, height, label in zip(bars,angles, heights, top_30['No. of times played'].astype(str)+ ' | ' +top_30['artist']):

    # Labels are rotated. Rotation must be specified in degrees :(
    rotation = np.rad2deg(angle)

    # Flip some labels upside down
    alignment = ""
    if angle >= np.pi/2 and angle < 3*np.pi/2:
        alignment = "right"
        rotation = rotation + 180
    else: 
        alignment = "left"

    # Finally add the labels
    ax.text(
        x=angle, 
        y=lowerLimit + bar.get_height() + labelPadding, 
        s=label, 
        ha=alignment, 
        va='center', 
        rotation=rotation, 
        rotation_mode="anchor") 

    
# for bar, angle, height, label in zip(bars,angles, heights, top_30['No. of times played']):

#     # Labels are rotated. Rotation must be specified in degrees :(
#     rotation = np.rad2deg(angle)

#     # Flip some labels upside down
#     alignment = ""
#     if angle >= np.pi/2 and angle < 3*np.pi/2:
#         alignment = "right"
#         rotation = rotation + 180
#     else: 
#         alignment = "left"

#     # Finally add the labels
#     ax.text(
#         x=angle, 
#         y= lowerLimit + bar.get_height() + labelPadding,
#         s=label, 
#         ha=alignment, 
#         va='center', 
#         rotation=rotation, 
#         rotation_mode="anchor",
#         color = 'white')
#         #weight = 'heavy' 


pdf_out.savefig()

In [23]:
pdf_out.close()